In [2]:
import os
import csv
import time
import requests
import vcfpy
from dotenv import load_dotenv
from tqdm import tqdm


In [3]:
load_dotenv()

VCF_DATA = os.getenv("VCF_DATA_")
VCF_INDEX = os.getenv("VCF_DATA_INDEX")
DATA_PATH = os.getenv("DATA_PATH")

print(VCF_DATA)


C:\Users\Noodl\Projects\Big_O\Hackathon\Rift2k26\Data\ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz


In [4]:
def open_vcf():
    return vcfpy.Reader.from_path(VCF_DATA)

vcf_reader = open_vcf()


In [5]:
clinical_db = {}

file_path = os.path.join(DATA_PATH, "clinicalVariants/clinicalVariants.tsv")

with open(file_path, encoding="utf-8") as f:

    reader = csv.DictReader(f, delimiter="\t")

    print("Columns:", reader.fieldnames)

    for row in reader:

        variant = row["variant"].strip()

        # keep ONLY rsIDs (VCF-compatible)
        if not variant.startswith("rs"):
            continue

        clinical_db.setdefault(variant, []).append({
            "gene": row["gene"],
            "drug": row["chemicals"],
            "phenotype": row["phenotypes"],
            "evidence": row["level of evidence"],
            "type": row["type"]
        })

print("Loaded clinical variants:", len(clinical_db))

Columns: ['variant', 'gene', 'type', 'level of evidence', 'chemicals', 'phenotypes']
Loaded clinical variants: 2844


In [6]:
def chunks(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


In [7]:
import json
import os

CACHE_FILE = os.path.join(DATA_PATH, "ensembl_variant_cache.json")


In [8]:
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        variant_cache = json.load(f)
else:
    variant_cache = {}

print("Cached variants:", len(variant_cache))


Cached variants: 0


In [9]:
missing_rsids = [
    rsid for rsid in clinical_db.keys()
    if rsid not in variant_cache
]

print("Need to fetch:", len(missing_rsids))


Need to fetch: 2844


In [14]:
TARGET_GENES = {
    "CYP2D6",
    "CYP2C19",
    "CYP2C9",
    "SLCO1B1",
    "TPMT",
    "DPYD"
}

filtered_clinical = {}

for rsid, anns in clinical_db.items():
    for a in anns:
        if a["gene"] in TARGET_GENES:
            filtered_clinical[rsid] = anns
            break

print("Filtered variants:", len(filtered_clinical))


Filtered variants: 123


In [15]:
rsids = list(filtered_clinical.keys())


In [16]:
from concurrent.futures import ThreadPoolExecutor
import requests

def fetch_variant(rsid):
    try:
        r = requests.get(
            f"https://rest.ensembl.org/variation/human/{rsid}",
            headers={"Accept":"application/json"},
            timeout=15
        )

        if not r.ok:
            return None

        data = r.json()

        return rsid, data.get("mappings", [])

    except:
        return None


with ThreadPoolExecutor(max_workers=8) as ex:
    results = list(ex.map(fetch_variant, rsids))


In [17]:
results

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ('rs3918290',
  [{'assembly_name': 'GRCh38',
    'seq_region_name': '1',
    'start': 97450058,
    'allele_string': 'C/A/G/T',
    'location': '1:97450058-97450058',
    'strand': 1,
    'ancestral_allele': 'C',
    'coord_system': 'chromosome',
    'end': 97450058}]),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ('rs1142345',
  [{'start': 18130687,
    'seq_region_name': '6',
    'strand': 1,
    'end': 18130687,
    'ancestral_allele': 'T',
    'allele_string': 'T/A/C/G',
    'location': '6:18130687-18130687',
    'assembly_name': 'GRCh38',
    'coord_system': 'chromosome'}]),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ('rs28371706',
  [{'end': 42129770,
    'seq_region_name': '22',
    'strand': 1,
    'coord_system': 'chromosom

In [ ]:
from tqdm import tqdm
import time
import requests

def chunks(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


for batch in tqdm(list(chunks(missing_rsids, 25))):

    try:
        r = requests.post(
            "https://rest.ensembl.org/variation/human",
            json={"ids": batch},
            headers={"Content-Type": "application/json"},
            timeout=60
        )

        if not r.ok:
            continue

        data = r.json()

        for rsid, info in data.items():

            positions = []

            for m in info.get("mappings", []):
                positions.append({
                    "chrom": m["seq_region_name"],
                    "pos": m["start"]
                })

            variant_cache[rsid] = positions

        # ⭐ polite delay (VERY IMPORTANT)
        time.sleep(0.5)

    except Exception as e:
        print("FAILED batch:", e)


  0%|          | 0/114 [00:00<?, ?it/s]

  1%|          | 1/114 [01:02<1:57:18, 62.29s/it]

FAILED batch: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=60)


  2%|▏         | 2/114 [02:03<1:55:13, 61.73s/it]

FAILED batch: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=60)


  3%|▎         | 3/114 [03:05<1:54:03, 61.65s/it]

FAILED batch: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=60)


In [ ]:
clinical_pos_db = {}

rsids = list(clinical_db.keys())

for batch in tqdm(list(chunks(rsids, 25))):

    try:
        r = requests.post(
            "https://rest.ensembl.org/variation/human",
            json={"ids": batch},
            headers={"Content-Type": "application/json"},
            timeout=60
        )

        if not r.ok:
            continue

        data = r.json()

        for rsid, info in data.items():

            if "mappings" not in info:
                continue

            for m in info["mappings"]:

                # keep only chr22 for now (CYP2D6 test)
                if m["seq_region_name"] != "22":
                    continue

                key = (m["seq_region_name"], m["start"])
                clinical_pos_db.setdefault(key, []).extend(
                    clinical_db[rsid]
                )

    except Exception as e:
        print("FAILED batch:", e)

print("Mapped positions:", len(clinical_pos_db))


  1%|          | 1/114 [01:01<1:55:33, 61.36s/it]

FAILED batch: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=60)


In [ ]:
gene_regions = {
    "CYP2D6": ("22", 42125962, 42131236),
    "CYP2C19": ("10", 96522437, 96612962),
    "CYP2C9": ("10", 96698402, 96761229),
    "SLCO1B1": ("12", 21176840, 21256333),
    "TPMT": ("6", 18128306, 18143127),
    "DPYD": ("1", 97450000, 97590000),
}


In [ ]:
vcf_reader = open_vcf()   # reopen reader!

hits = []

for gene, (chrom, start, end) in gene_regions.items():

    print(f"\nScanning {gene}")

    for record in vcf_reader.fetch(chrom, start, end):

        key = (record.CHROM, record.POS)

        if key not in clinical_pos_db:
            continue

        for ann in clinical_pos_db[key]:

            hit = {
                "gene": gene,
                "pos": record.POS,
                "ref": record.REF,
                "alt": [str(a.value) for a in record.ALT],
                "drug": ann["drug"],
                "evidence": ann["evidence"]
            }

            hits.append(hit)
            print("CLINICAL HIT:", hit)

print("\nTotal hits:", len(hits))
